<a href="https://colab.research.google.com/github/MyDearGreatTeacher/AI4high/blob/master/UsingColab_git.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colab常用技巧

https://mattwang44.github.io/en/articles/colab/

## 上傳檔案到Colab與下載Colab上的檔案

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
from google.colab import files
files.download('WestWorld.png')

# 開啟GitHub上的Notebook

# 下載Github上的程式及模組並能順利執行

20190506

In [0]:
!git clone https://github.com/oreilly-japan/deep-learning-from-scratch.git

In [0]:
!ls deep-learning-from-scratch/dataset

In [0]:
!cat deep-learning-from-scratch/ch01/hungry.py

In [0]:
!python deep-learning-from-scratch/ch01/hungry.py

## 切換到不同目錄執行目錄底下的程式

In [0]:
import os
os.chdir("deep-learning-from-scratch")

!ls

ch01  ch03  ch05  ch07	common	 LICENSE.md
ch02  ch04  ch06  ch08	dataset  README.md


In [0]:
import os
os.chdir("ch06")

!python batch_norm_gradient_check.py

Done
Done
Done
Done
Converting train-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting train-labels-idx1-ubyte.gz to NumPy Array ...
Done
Converting t10k-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting t10k-labels-idx1-ubyte.gz to NumPy Array ...
Done
Creating pickle file ...
Done!
W1:0.0
b1:0.0
gamma1:0.0
beta1:0.0
W2:0.0
b2:0.0
gamma2:0.0
beta2:0.05884638085620342
W3:0.0
b3:1.7990402263745597e-07


# Run Tensorboard with Google Colab



Quick guide to run TensorBoard in Google Colab

https://www.dlology.com/blog/quick-guide-to-run-tensorboard-in-google-colab/




Step1. 在Colab下下載Ngrok

In [1]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-05-06 15:43:12--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.203.66.95, 34.206.36.121, 34.206.130.40, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.203.66.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14991793 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  14.30M  18.0MB/s    in 0.8s    

2019-05-06 15:43:13 (18.0 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [14991793/14991793]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


Step2.指定訓練過程的紀錄檔(logdir)要存放在virtual machine下的哪個位置，並開啟tensorboard service(port 6006)

In [0]:
LOG_DIR = './log'
get_ipython().system_raw(
'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
.format(LOG_DIR)
)

Step3.開啟ngrok service，並綁定port 6006(tensorboard)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

Step4.產生一組網址，這組接下來只要從外部連到這組網址，就會導向virtual machine的port 6006，也就是colab裝置內的tensorboard了！

In [4]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
"import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"


http://62d5d061.ngrok.io


In [6]:
'''Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import TensorBoard

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])


tbCallBack = TensorBoard(log_dir='./log', histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         batch_size=batch_size,
                         write_images=True)

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[tbCallBack])
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


11493376/11490434 [==============================] - 1s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 165s 3ms/step - loss: 0.2701 - acc: 0.9165 - val_loss: 0.0575 - val_acc: 0.9823
Epoch 2/12
60000/60000 [==============================] - 164s 3ms/step - loss: 0.0876 - acc: 0.9742 - val_loss: 0.0386 - val_acc: 0.9877
Epoch 3/12
60000/60000 [==============================] - 164s 3ms/step - loss: 0.0649 - acc: 0.9804 - val_loss: 0.0316 - val_acc: 0.9896
Epoch 4/12
60000/60000 [==============================] - 165s 3ms/step - loss: 0.0531 - acc: 0.9840 - val_loss: 0.0312 - val_acc: 0.9897
Epoch 5/12
60000/60000 [=

In [7]:
!ps aux | grep tensorboard

root         162  0.5  2.2 881852 306576 ?       Sl   15:43   0:16 /usr/bin/python2 /usr/local/bin/tensorboard --logdir ./log --host 0.0.0.0 --port 6006
root         897  0.0  0.0  39196  6356 ?        S    16:33   0:00 /bin/bash -c ps aux | grep tensorboard


In [8]:
!ps aux | grep ngrok

root         169  0.2  0.2 241928 39648 ?        Sl   15:43   0:07 ./ngrok http 6006
root         900  0.0  0.0  39192  6520 ?        S    16:33   0:00 /bin/bash -c ps aux | grep ngrok
root         902  0.0  0.0  38568  5484 ?        S    16:33   0:00 grep ngrok


# 其他測試

In [0]:
!mkdir -p Drive
!google-drive-ocamlfuse Drive

/bin/bash: google-drive-ocamlfuse: command not found


In [0]:
!git clone https://github.com/oreilly-japan/deep-learning-from-scratch.git

fatal: destination path 'deep-learning-from-scratch' already exists and is not an empty directory.


In [0]:
!PATH=deep-learning-from-scratch

In [0]:
!python deep-learning-from-scratch/ch06/batch_norm_gradient_check.py

Traceback (most recent call last):
  File "deep-learning-from-scratch/ch06/batch_norm_gradient_check.py", line 5, in <module>
    from dataset.mnist import load_mnist
ModuleNotFoundError: No module named 'dataset'


In [0]:
!ls

deep-learning-from-scratch  sample_data


In [0]:
!cd deep-learning-from-scratch

In [0]:
!ls deep-learning-from-scratch/dataset

__init__.py  lena_gray.png  lena.png  mnist.py


In [0]:
!ls deep-learning-from-scratch/ch01

hungry.py  img_show.py	man.py	simple_graph.py  sin_cos_graph.py  sin_graph.py


In [0]:
!python deep-learning-from-scratch/ch01/simple_graph.py

<Figure size 640x480 with 1 Axes>


In [0]:
!python deep-learning-from-scratch/ch01/hungry.py

I'm hungry!


In [0]:
!python deep-learning-from-scratch/ch04/gradient_method.py

<Figure size 640x480 with 1 Axes>
